In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import font_manager, rc
import platform
import statsmodels.formula.api as smf

if platform.system() == 'Windows':
# windows 인 경우
    font_name = font_manager.FontProperties(fname="c:\Windows\Fonts\malgun.ttf").get_name()
    rc('font', family=font_name)
else: 
# mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
matplotlib.rcParams['axes.unicode_minus'] = False
from patsy import dmatrices
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline
import time
import itertools 
from sklearn.model_selection import train_test_split

In [10]:
data = pd.read_csv("1,2,3월_2.csv")

In [11]:
data = data.drop(['Unnamed: 0'], axis = 1, inplace = False)
data["연령대"] = data["연령대"].str.rstrip("s").astype("int")

In [12]:
data = data[((data['업종명'] == '일반 음식점업') | (data['업종명'] == "전시 및 행사 대행업")
                  |(data["업종명"] == '관광업') | (data["업종명"] == "카페")| (data["업종명"] == "스포츠 및 레크레이션 용품 임대업"))]

In [13]:
# 업종별 데이터 분류
df1 = data[(data['업종명'] == '카페')]
df2 = data[(data['업종명'] == '스포츠 및 레크레이션 용품 임대업')]
df3 = data[(data['업종명'] == '일반 음식점업')]
df4 = data[(data['업종명'] == '전시 및 행사 대행업')]
df5 = data[(data['업종명'] == '관광업')]

In [14]:
df1

,년월,업종명,연령대,성별,생애주기,고객수,금액,건수
0,201901,카페,20,2,1,3,24890,3
1,201901,카페,20,2,1,3,17800,3
2,201901,카페,30,2,2,3,28500,4
3,201901,카페,20,1,1,3,27900,4
4,201901,카페,40,1,3,3,20600,3
...,...,...,...,...,...,...,...,...
223758,202003,카페,30,1,2,3,29600,5
223759,202003,카페,40,1,3,3,21800,7
223760,202003,카페,20,2,1,3,7500,3
223761,202003,카페,20,1,1,6,54900,7


# cafe
## 3가지 방법 모두 연령대, 생애주기, 고객수, 건수를 변수로 저장

In [15]:
df1 = df1[["년월","업종명","연령대","성별","생애주기","고객수","건수","금액"]]

In [16]:
# 전진 선택법
variables = df1.columns[2:-1].tolist() # 설명변수 설정

y =df1["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df1[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df1[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\

In [17]:
selected_variables

['고객수', '연령대', '건수', '생애주기']

In [18]:
# 후진 제거법
variables = df1.columns[2:-1].tolist() # 설명변수 설정

y =df1["금액"] # 반응변수 설정
selected_variables= variables # 초기에는 모든 변수 선택
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    X = sm.add_constant(df1[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:]
    max_pval = p_vals.max()
    if max_pval >= sl_remove:
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df1[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [19]:
selected_variables

['연령대', '생애주기', '고객수', '건수']

In [20]:
# 단계적 선택법 
variables = df1.columns[2:-1].tolist() # 설명변수 설정

y =df1["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df1[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        while len(selected_variables) > 0:
            selected_X = df1[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:]
            max_pval = selected_pval.max()
            if max_pval >= sl_remove:
                remove_variable = selected_pval.idxmax()
                selected_variabels.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df1[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'obj

In [21]:
selected_variables

['고객수', '연령대', '건수', '생애주기']

In [22]:
model1 = smf.ols(formula = "금액 ~ 고객수+연령대+건수+생애주기", data = df1)
result1 = model1.fit()
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     금액   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                 4.922e+05
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        17:38:01   Log-Likelihood:            -5.2200e+05
No. Observations:               36070   AIC:                         1.044e+06
Df Residuals:                   36065   BIC:                         1.044e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.517e+05   7538.664    -20.129      0.000   -1.67e+05   -1.37e+05
고객수         1.074e+04     83.836    128.136      0.000    1.06e+04    1.09e+04
연령대         3605.9034    521.262      6.918      0.000    2584.214    4627.593
건수           777.5586     52.116     14.920      0.000     675.409     879.708
생애주기        1.494e+04   5395.953      2.769      0.006    4367.728    2.55e+04
==============================================================================
Omnibus:                    60759.658   Durbin-Watson:                   1.588
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        472944375.127
Skew:                          10.632   Prob(JB):                         0.00
Kurtosis:                     563.564   Cond. No.                     1.86e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.86e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# sport
## 세가지 방법 모두 모든 변수 선택

In [23]:
df2 = df2[["년월","업종명","연령대","성별","생애주기","고객수","건수","금액"]]

In [24]:
# 전진 선택법
variables = df2.columns[2:-1].tolist() # 설명변수 설정

y =df2["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df2[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df2[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\

In [25]:
selected_variables

['고객수', '생애주기', '건수', '성별', '연령대']

In [26]:
# 후진 제거법
variables = df2.columns[2:-1].tolist() # 설명변수 설정

y =df2["금액"] # 반응변수 설정
selected_variables= variables # 초기에는 모든 변수 선택
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    X = sm.add_constant(df2[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:]
    max_pval = p_vals.max()
    if max_pval >= sl_remove:
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df2[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [27]:
selected_variables

['연령대', '성별', '생애주기', '고객수', '건수']

In [28]:
# 후진 제거법
variables = df2.columns[2:-1].tolist() # 설명변수 설정

y =df2["금액"] # 반응변수 설정
selected_variables= variables # 초기에는 모든 변수 선택
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    X = sm.add_constant(df2[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:]
    max_pval = p_vals.max()
    if max_pval >= sl_remove:
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df2[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [29]:
selected_variables

['연령대', '성별', '생애주기', '고객수', '건수']

In [30]:
# 단계적 선택법 
variables = df2.columns[2:-1].tolist() # 설명변수 설정

y =df2["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df2[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        while len(selected_variables) > 0:
            selected_X = df2[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:]
            max_pval = selected_pval.max()
            if max_pval >= sl_remove:
                remove_variable = selected_pval.idxmax()
                selected_variabels.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df2[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'obj

In [31]:
selected_variables

['고객수', '생애주기', '건수', '성별', '연령대']

In [32]:
model2 = smf.ols(formula = "금액 ~ 연령대+생애주기+성별+고객수+건수", data = df2)
result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     금액   R-squared:                       0.937
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                 3.342e+04
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        17:38:10   Log-Likelihood:            -1.8206e+05
No. Observations:               11301   AIC:                         3.641e+05
Df Residuals:                   11295   BIC:                         3.642e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.773e+06   9.55e+04    -18.562      0.000   -1.96e+06   -1.59e+06
연령대           1.2e+04   4420.395      2.715      0.007    3336.932    2.07e+04
생애주기        3.035e+05   4.75e+04      6.391      0.000     2.1e+05    3.97e+05
성별          1.522e+05    4.6e+04      3.308      0.001     6.2e+04    2.42e+05
고객수         6.759e+04   2558.313     26.420      0.000    6.26e+04    7.26e+04
건수          4.801e+04   2048.189     23.441      0.000     4.4e+04     5.2e+04
==============================================================================
Omnibus:                     4540.931   Durbin-Watson:                   1.443
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           646298.645
Skew:                          -0.880   Prob(JB):                         0.00
Kurtosis:                      40.006   Cond. No.                         575.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Food
## 세가지 방법 모두 모든 변수 선택

In [34]:
df3 = df3[["년월","업종명","연령대","성별","생애주기","고객수","건수","금액"]]

In [36]:
# 전진 선택법
variables = df3.columns[2:-1].tolist() # 설명변수 설정

y =df3["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df3[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df3[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\

In [37]:
selected_variables

['고객수', '건수', '연령대', '성별', '생애주기']

In [38]:
# 후진 제거법
variables = df3.columns[2:-1].tolist() # 설명변수 설정

y =df3["금액"] # 반응변수 설정
selected_variables= variables # 초기에는 모든 변수 선택
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    X = sm.add_constant(df3[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:]
    max_pval = p_vals.max()
    if max_pval >= sl_remove:
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df3[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [39]:
selected_variables

['연령대', '성별', '생애주기', '고객수', '건수']

In [40]:
# 단계적 선택법 
variables = df3.columns[2:-1].tolist() # 설명변수 설정

y =df3["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df3[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        while len(selected_variables) > 0:
            selected_X = df3[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:]
            max_pval = selected_pval.max()
            if max_pval >= sl_remove:
                remove_variable = selected_pval.idxmax()
                selected_variabels.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df3[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'obj

In [41]:
selected_variables

['고객수', '건수', '연령대', '성별', '생애주기']

In [42]:
model3 = smf.ols(formula = "금액 ~ 연령대+성별+생애주기+고객수+건수", data = df3)
result3 = model3.fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     금액   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.412e+06
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        17:38:22   Log-Likelihood:            -2.7409e+06
No. Observations:              165688   AIC:                         5.482e+06
Df Residuals:                  165682   BIC:                         5.482e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.843e+05   4.04e+04    -11.978      0.000   -5.64e+05   -4.05e+05
연령대         1.276e+04   1912.351      6.673      0.000    9012.480    1.65e+04
성별         -3.357e+05   1.89e+04    -17.733      0.000   -3.73e+05   -2.99e+05
생애주기        1.014e+05   1.97e+04      5.160      0.000    6.29e+04     1.4e+05
고객수        -8391.8081    221.430    -37.898      0.000   -8825.806   -7957.811
건수          3.844e+04    146.515    262.340      0.000    3.81e+04    3.87e+04
==============================================================================
Omnibus:                   141498.061   Durbin-Watson:                   1.622
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       3236662438.967
Skew:                          -2.296   Prob(JB):                         0.00
Kurtosis:                     687.698   Cond. No.                     4.23e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Display
## 세가지 방법 모두 고객수, 연령대, 건수, 성별 선택

In [43]:
df4 = df4[["년월","업종명","연령대","성별","생애주기","고객수","건수","금액"]]

In [44]:
# 전진 선택법
variables = df4.columns[2:-1].tolist() # 설명변수 설정

y =df4["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df4[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df4[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\

In [45]:
selected_variables

['고객수', '연령대', '건수', '성별']

In [46]:
# 후진 제거법
variables = df4.columns[2:-1].tolist() # 설명변수 설정

y =df4["금액"] # 반응변수 설정
selected_variables= variables # 초기에는 모든 변수 선택
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    X = sm.add_constant(df4[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:]
    max_pval = p_vals.max()
    if max_pval >= sl_remove:
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df4[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [47]:
selected_variables

['연령대', '성별', '고객수', '건수']

In [48]:
# 단계적 선택법 
variables = df4.columns[2:-1].tolist() # 설명변수 설정

y =df4["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df4[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        while len(selected_variables) > 0:
            selected_X = df4[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:]
            max_pval = selected_pval.max()
            if max_pval >= sl_remove:
                remove_variable = selected_pval.idxmax()
                selected_variabels.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df4[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'obj

In [49]:
selected_variables

['고객수', '연령대', '건수', '성별']

In [50]:
model4 = smf.ols(formula = "금액 ~ 연령대+성별+고객수+건수", data = df4)
result4 = model4.fit()
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     금액   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     6630.
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        17:38:29   Log-Likelihood:                -87582.
No. Observations:                6217   AIC:                         1.752e+05
Df Residuals:                    6212   BIC:                         1.752e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.017e+05    1.9e+04    -21.128      0.000   -4.39e+05   -3.64e+05
연령대         4646.8870    338.132     13.743      0.000    3984.032    5309.742
성별          5.291e+04   8179.572      6.468      0.000    3.69e+04    6.89e+04
고객수         6.094e+04   1540.066     39.573      0.000    5.79e+04     6.4e+04
건수         -1.171e+04   1328.259     -8.813      0.000   -1.43e+04   -9102.573
==============================================================================
Omnibus:                     7268.299   Durbin-Watson:                   1.177
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3078403.270
Skew:                           5.602   Prob(JB):                         0.00
Kurtosis:                     111.436   Cond. No.                         212.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Tour
## 3가지 방법 모두 고객수, 건수, 성별 변수 선택

In [51]:
df5 = df5[["년월","업종명","연령대","성별","생애주기","고객수","건수","금액"]]

In [52]:
# 전진 선택법
variables = df5.columns[2:-1].tolist() # 설명변수 설정

y =df5["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df5[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df5[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [53]:
selected_variables

['고객수', '건수', '성별']

In [54]:
# 후진 제거법
variables = df5.columns[2:-1].tolist() # 설명변수 설정

y =df5["금액"] # 반응변수 설정
selected_variables= variables # 초기에는 모든 변수 선택
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    X = sm.add_constant(df5[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:]
    max_pval = p_vals.max()
    if max_pval >= sl_remove:
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df5[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [55]:
selected_variables

['성별', '고객수', '건수']

In [56]:
# 단계적 선택법 
variables = df5.columns[2:-1].tolist() # 설명변수 설정

y =df5["금액"] # 반응변수 설정
selected_variables=[] # 선택된 변수
sl_enter = 0.05
sl_remove = 0.05

sv_per_step = [] # 단계별 선택된 변수
adjusted_r_squared = [] # 단계별 수정된 결정계수
steps = []
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index = remainder)
    for col in remainder:
        X = df5[selected_variables+[col]]  
        X = sm.add_constant(X) 
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
   
    min_pval = pval.min()
    if min_pval < sl_enter:
        selected_variables.append(pval.idxmin())
        while len(selected_variables) > 0:
            selected_X = df5[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:]
            max_pval = selected_pval.max()
            if max_pval >= sl_remove:
                remove_variable = selected_pval.idxmax()
                selected_variabels.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y, sm.add_constant(df5[selected_variables]))
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
C:\Users\LG\anaconda3\envs\tens_2\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'obj

In [57]:
selected_variables

['고객수', '건수', '성별']

In [58]:
# 세가지 변수를 이용한 회귀 모델 구현
model5 = smf.ols(formula = "금액 ~ 성별+고객수+건수", data = df5)
result5 = model5.fit()
result5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     금액   R-squared:                       0.328
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     856.0
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        17:38:35   Log-Likelihood:                -83873.
No. Observations:                5264   AIC:                         1.678e+05
Df Residuals:                    5260   BIC:                         1.678e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.251e+04   8.68e+04     -0.259      0.795   -1.93e+05    1.48e+05
성별         -1.526e+05   5.66e+04     -2.696      0.007   -2.64e+05   -4.16e+04
고객수         2.622e+05   9355.977     28.023      0.000    2.44e+05    2.81e+05
건수         -1.005e+05   5994.755    -16.764      0.000   -1.12e+05   -8.87e+04
==============================================================================
Omnibus:                     7691.317   Durbin-Watson:                   1.312
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4253981.096
Skew:                           8.623   Prob(JB):                         0.00
Kurtosis:                     141.194   Cond. No.                         99.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""